In [41]:
# !/use/bin/env python
# encoding:utf-8
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.model_selection import train_test_split
import math
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
import easy_excel
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import *
import sklearn.ensemble
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB  
import subprocess
from sklearn.utils import shuffle
import itertools
from sklearn.ensemble import GradientBoostingClassifier
import sys
from imblearn.combine import SMOTETomek
inputname="m6a_data.txt"
outputname="preprocessingFile_m6a.fasta"
path="/home02/chenhuangrong/december_data/"

In [24]:
def read_fasta_file(path):
    '''
    used for load fasta data and transformd into numpy.array format
    '''
    fh=open(m6a_benchmark_dataset)
    seq=[]
    for line in fh:
        if line.startswith('>'):
            continue
        else:
            seq.append(line.replace('\r','').replace('\n',''))
    fh.close()
    matrix_data=np.array([list(e) for e in seq])
    return matrix_data

In [25]:
def Comparing_score(query_sequence,original_sequence):
    
    score=0
    for index,data in enumerate(query_sequence):
        if data==original_sequence[index]:
            score=score+2
        else:
            score=score-1
    return score


In [26]:
def generating_one_column(matrix_data):
    whole_comparison_score=[]
    the_begin_of_index=len(matrix_data)/2
    for index_1,data_1 in enumerate(matrix_data):
        one_line_comparison_score=np.zeros(len(matrix_data))
        mark_origin_label=np.zeros(len(matrix_data))
        for index_2,data_2 in enumerate(matrix_data):
            if index_1!=index_2:
                one_line_comparison_score[index_1]=-100
                if index_2<the_begin_of_index:
                    mark_origin_label[index_2]=1
                one_line_comparison_score[index_2]=Comparing_score(data_1,data_2)
        temp=[]
        temp=[one_line_comparison_score,mark_origin_label]
        whole_comparison_score.append(temp)
    return whole_comparison_score

In [151]:
def generating_features(matrix_data,K_list):
    matrix=generating_one_column(matrix_data)
    print np.asarray(matrix).shape
    whole_=[]
    line=[]
    for index,data in enumerate(matrix):
        idx=np.argsort(data[0])[::-1]
        idx=idx[xrange(K_list)]
        data[1]=pd.DataFrame(data[1])
        datas=data[1].iloc[idx]
        datas=datas.values
        all_value=sum(datas)
        if index>=(len(matrix_data)//2):
            all_value=K_list-all_value
        if (index<(len(matrix_data)//2) and all_value>=17):
            line.append(["".join(matrix_data[index]),1])
        if (index>=(len(matrix_data)//2) and all_value>=13):
            line.append(["".join(matrix_data[index]),0])
    print "whole length is :{}".format(len(line))
    return line

In [152]:

m6a_benchmark_dataset=path+inputname
matrix_data=read_fasta_file(m6a_benchmark_dataset)
final_feature_matrix=generating_features(matrix_data,30)
pd.DataFrame(final_feature_matrix[0]).to_csv(path+outputname,header=None,index=False)




(2614, 2, 2614)
whole length is :1903


In [153]:
def save_to_file(file_name, contents):
    fh = open(file_name, 'w')
    for line in contents:
        fh.write(str(line)+'\n')
    fh.close()
save_to_file(path+outputname,final_feature_matrix)

In [154]:
def load_from_file(filename):
    fh= open(filename,'r')
    lines =list(fh.readlines())
    fh.close()
    X=[]
    Y=[]
    for line in lines:
        line= line.replace('\n',"").replace('[',"")
        line= line.replace(']',"").replace('\'',"")
        line_x= line.split(',')[0]
        line_y= line.split(',')[1]
        X.append(line_x)
        Y.append(int(line_y))
    return X,np.asarray(Y)

In [256]:
samples,labels=load_from_file(path+outputname)
negative_samples_num=len(samples)-sum(labels)
positive_samples_num=sum(labels)
print "original_positive_samples_num :{}".format(positive_samples_num)
print "original_negative_samples_num :{}".format(negative_samples_num)
if negative_samples_num >=positive_samples_num:
    negative_samples=np.random.choice(samples[positive_samples_num:],positive_samples_num, replace=False)
    negative_lables=np.random.choice(labels[positive_samples_num:],positive_samples_num, replace=False)
    positive_samples=samples[0:positive_samples_num]
    positive_labels=labels[0:positive_samples_num]
else:
    positive_samples=np.random.choice(samples[0:positive_samples_num],negative_samples_num, replace=False)
    positive_labels=np.random.choice(labels[0:positive_samples_num],negative_samples_num, replace=False)
    negative_samples=samples[positive_samples_num:]
    negative_labels=labels[positive_samples_num:]
print "final_positive_samples_num :{}".format(len(positive_samples))
print "final_negative_samples_num :{}".format(len(negative_samples))
XX=positive_samples
XX.extend(negative_samples)
save_to_file(path+"final_preprocessing.fasta",XX)
    

original_positive_samples_num :936
original_negative_samples_num :967
final_positive_samples_num :936
final_negative_samples_num :936
